In [2]:
import pandas as pd
import numpy as np


In [3]:
traffic_df = pd.read_csv('/home/karthiktiwari/Data/Dataset/Train.csv')

In [4]:
traffic_df.head()

,Holiday,Temperature,Rainfall_last_hour,Snowfall_last_hour,Cloud_Cover,Weather,Weather_Desc,TimeStamp,Date,Traffic_Vol
0,NaN,289.28,0.0,0.0,40,Cloudy skies,Partly cloudy skies,02/10/08 9:00,02/10/08,5555
1,NaN,290.26,0.0,0.0,75,Cloudy skies,Fragmented clouds,02/10/08 10:00,02/10/08,4525
2,NaN,290.28,0.0,0.0,90,Cloudy skies,Full cloud cover,02/10/08 11:00,02/10/08,4772
3,NaN,290.33,0.0,0.0,90,Cloudy skies,Full cloud cover,02/10/08 12:00,02/10/08,5031
4,NaN,292.14,0.0,0.0,75,Cloudy skies,Fragmented clouds,02/10/08 13:00,02/10/08,4928


In [5]:
np.max(traffic_df['Traffic_Vol'])

7289

In [6]:
#Number of holidays
np.sum(traffic_df['Holiday']).item(), len(np.unique(traffic_df['Date']))

(50.0, 1768)

In [7]:
traffic_df['Holiday'].fillna(0, inplace=True)

/tmp/ipykernel_3632/2267864254.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  traffic_df['Holiday'].fillna(0, inplace=True)


In [8]:
# Data points grouped by date column
# Intent is to use a seq2seq model for prediction using context



In [9]:
from sklearn.preprocessing import LabelEncoder
# Convert categorical variables to numerical
label_encoders = {}
for col in ['Weather', 'Weather_Desc']:
    encoder = LabelEncoder()
    traffic_df[col] = encoder.fit_transform(traffic_df[col])
    label_encoders[col] = encoder

# Sort the data by date
traffic_df['Date'] = pd.to_datetime(traffic_df['Date'])

# Include 'Date' column in features
X = traffic_df[['Date', 'Holiday', 'Temperature', 'Rainfall_last_hour', 'Snowfall_last_hour', 'Cloud_Cover', 'Weather', 'Weather_Desc']]
y = traffic_df['Traffic_Vol']

# Prepare data for sequence-to-sequence model
X_seq = X.groupby(pd.to_datetime(X['Date']).dt.date).apply(lambda x: x.values)
y_seq = y.groupby(pd.to_datetime(X['Date']).dt.date).apply(lambda x: x.values)


/tmp/ipykernel_3632/1441635868.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  traffic_df['Date'] = pd.to_datetime(traffic_df['Date'])


In [10]:
np.array(X.loc[0][1:])

array([0.0, 289.28, 0.0, 0.0, 40, 3, 19], dtype=object)

In [11]:
max_len = 0
for date, rows in X_seq.items():
    if rows.shape[0]>max_len:
        max_len = rows.shape[0]
max_len

24

In [12]:
import torch
from torch.utils.data import Dataset


class TrafficVolumeDataset(Dataset):
    def __init__(self, X_seq, y_seq):
        self.X_seq = [x[:, 1:] for x in X_seq]
        self.y_seq = y_seq

    def __len__(self):
        return len(self.X_seq)

    def __getitem__(self, index):
        X = self.X_seq[index]
        y = self.y_seq[index]
        return np.float32(X), np.float32(y)
        # return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [13]:
dataset = TrafficVolumeDataset(X_seq, y_seq)


def collate_fn(batch):
    X, y = zip(*batch)
    
    # Get the maximum sequence length
    max_length = max(len(x) for x in X)
    
    
    # Pad the sequences to the maximum length
    X_padded = torch.nn.utils.rnn.pad_sequence([torch.from_numpy(x) for x in X], batch_first=True, padding_value=0)
    y_padded = torch.nn.utils.rnn.pad_sequence([torch.from_numpy(y_i) for y_i in y], batch_first=True, padding_value=0)
    
    # Get the sequence lengths
    lengths = torch.tensor([len(x) for x in X])
    
    return X_padded, y_padded, lengths

dataloader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=True, collate_fn=collate_fn, drop_last=True)

for x_samp, y_samp, lengths in dataloader:
    print(x_samp[0], y_samp[0])
    break

tensor([[  0.0000, 252.8000,   0.0000,   0.0000,   2.0000,   2.0000,   3.0000],
        [  0.0000, 251.4900,   0.0000,   0.0000,   2.0000,   2.0000,   3.0000],
        [  0.0000, 251.6600,   0.0000,   0.0000,   1.0000,   2.0000,   3.0000],
        [  0.0000, 251.5400,   0.0000,   0.0000,  76.0000,   3.0000,   9.0000],
        [  0.0000, 250.8400,   0.0000,   0.0000,  75.0000,   3.0000,   9.0000],
        [  0.0000, 250.1100,   0.0000,   0.0000,  90.0000,   3.0000,  10.0000],
        [  0.0000, 250.1400,   0.0000,   0.0000,  40.0000,   3.0000,  19.0000],
        [  0.0000, 249.3000,   0.0000,   0.0000,   1.0000,   2.0000,   3.0000],
        [  0.0000, 249.5400,   0.0000,   0.0000,   1.0000,   2.0000,   3.0000],
        [  0.0000, 251.6200,   0.0000,   0.0000,   2.0000,   2.0000,   3.0000],
        [  0.0000, 252.9900,   0.0000,   0.0000,   1.0000,   2.0000,   3.0000],
        [  0.0000, 255.9300,   0.0000,   0.0000,   1.0000,   2.0000,   3.0000],
        [  0.0000, 256.5800,   0.0000,  

/tmp/ipykernel_3632/1669368327.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = self.y_seq[index]


In [14]:
x_samp.to('cuda')

tensor([[[  0.0000, 252.8000,   0.0000,  ...,   2.0000,   2.0000,   3.0000],
         [  0.0000, 251.4900,   0.0000,  ...,   2.0000,   2.0000,   3.0000],
         [  0.0000, 251.6600,   0.0000,  ...,   1.0000,   2.0000,   3.0000],
         ...,
         [  0.0000, 256.5400,   0.0000,  ...,  75.0000,   3.0000,   9.0000],
         [  0.0000, 255.6600,   0.0000,  ...,  90.0000,   3.0000,  10.0000],
         [  0.0000, 255.9800,   0.0000,  ...,  20.0000,   3.0000,  20.0000]],

        [[  0.0000, 286.8200,   0.0000,  ...,   0.0000,   2.0000,   3.0000],
         [  0.0000, 286.2600,   0.0000,  ...,   0.0000,   2.0000,   3.0000],
         [  0.0000, 285.8800,   0.0000,  ...,   0.0000,   2.0000,   3.0000],
         ...,
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]],

        [[  0.0000, 272.6100,   0.0000,  ...

In [15]:
import torch
import torch.nn as nn

class TrafficVolumeModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_normal_(module.weight)
            nn.init.constant_(module.bias, 0)
        elif isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if 'weight' in name:
                    nn.init.xavier_normal_(param)
                elif 'bias' in name:
                    nn.init.constant_(param, 0)

    def forward(self, x, lengths):

        # Pass through the LSTM
        output, (h_n, c_n) = self.lstm(x)


        # Initialize tensor to store predictions
        fc_preds = []
        
        # Process each timestep separately
        for timestep in range(output.size(1)):
            timestep_pred = self.fc(output[:, timestep, :])
            fc_preds.append(timestep_pred)
        
        # Concatenate all predictions
        fc_preds = torch.stack(fc_preds, dim=1)
        
        return fc_preds.reshape(fc_preds.shape[0], fc_preds.shape[1])
    

# Hyperparameters
input_size = X_seq[0].shape[1]-1
hidden_size = 256
num_layers = 1
output_size = 1

# Initialize the model
model = TrafficVolumeModel(input_size, hidden_size, num_layers, output_size)


model.apply(model._init_weights)
model.to('cuda')
# Define the loss function and optimizer
criterion = nn.SmoothL1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

from tqdm import tqdm
# Training loop
num_epochs = 15
for epoch in tqdm(range(num_epochs)):
    for inputs, logits, lengths in dataloader:
        inputs = inputs.to('cuda')
        logits = logits.to('cuda')
        # Forward pass
        output = model(inputs, lengths)
        loss = criterion(output.squeeze(), logits)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)


    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/tmp/ipykernel_3632/2078800113.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  input_size = X_seq[0].shape[1]-1
  0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipykernel_3632/1669368327.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = self.y_seq[index]
 13%|█▎        | 2/15 [00:00<00:03,  3.35it/s]

Epoch [1/15], Loss: 3014.1567
Epoch [2/15], Loss: 2949.1440


 27%|██▋       | 4/15 [00:01<00:02,  4.66it/s]

Epoch [3/15], Loss: 2947.2986
Epoch [4/15], Loss: 2973.8999


 40%|████      | 6/15 [00:01<00:01,  5.52it/s]

...

Epoch [10/15], Loss: 2873.4519
Epoch [11/15], Loss: 2974.4097


 87%|████████▋ | 13/15 [00:02<00:00,  6.09it/s]

Epoch [12/15], Loss: 2999.6726
Epoch [13/15], Loss: 3026.5967


100%|██████████| 15/15 [00:02<00:00,  5.34it/s]

Epoch [14/15], Loss: 2988.0967
Epoch [15/15], Loss: 2965.6160


In [16]:
output.shape, logits.shape

(torch.Size([256, 24]), torch.Size([256, 24]))

In [17]:
lstm1 = nn.LSTM(7, 32, 1, batch_first=True)
fc1 = nn.Linear(32, 1)

In [18]:
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# import numpy as np

# # Flatten the sequence data if required or use time-series aggregation techniques for each sequence
# X_flattened = np.array([np.array(X.loc[i][1:], dtype=np.float32) for i in range(len(X))])
# y_flattened = np.float32(y)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_flattened, y_flattened, test_size=0.2, random_state=42)

# # Define XGBoost model
# xgb_model = xgb.XGBRegressor(
#     n_estimators=1000,  # Number of boosting rounds
#     learning_rate=0.01,  # Step size shrinkage to prevent overfitting
#     max_depth=6,  # Maximum depth of a tree
#     subsample=0.8,  # Subsample ratio of the training instance
#     colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree
#     random_state=42
# )

# # Train the model
# xgb_model.fit(
#     X_train, y_train,
#     eval_set=[(X_test, y_test)]
# )

# # Evaluate the model
# y_pred = xgb_model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print(f"Mean Squared Error: {mse:.4f}")


In [36]:
test_df = pd.read_csv('/home/karthiktiwari/Data/Dataset/Test.csv')
test_df['Holiday'].fillna(0, inplace=True)

/tmp/ipykernel_3632/3375765300.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Holiday'].fillna(0, inplace=True)


In [37]:
label_encoders = {}
for col in ['Weather', 'Weather_Desc']:
    encoder = LabelEncoder()
    test_df[col] = encoder.fit_transform(test_df[col])
    label_encoders[col] = encoder
test_df['Traffic Vol'] = np.zeros(len(test_df))

# Include 'Date' column in features
X_test = test_df[['Date', 'Holiday', 'Temperature', 'Rainfall_last_hour', 'Snowfall_last_hour', 'Cloud_Cover', 'Weather', 'Weather_Desc']]
y_test = test_df['Traffic_Vol']

# Prepare data for sequence-to-sequence model
X_test = X_test.groupby(pd.to_datetime(X['Date']).dt.date).apply(lambda x: x.values)
y_test = y_test.groupby(pd.to_datetime(X['Date']).dt.date).apply(lambda x: x.values)


In [38]:
test_data = TrafficVolumeDataset(X_test, y_test)

In [39]:
test_data[0]

/tmp/ipykernel_3632/1669368327.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = self.y_seq[index]


(array([[  0.  , 297.69,   0.  ,   0.  ,  90.  ,   8.  ,   1.  ],
        [  0.  , 296.09,   0.  ,   0.  ,  90.  ,   8.  ,   1.  ],
        [  0.  , 294.16,   0.  ,   0.  ,  40.  ,   7.  ,  16.  ],
        [  0.  , 292.52,   0.  ,   0.  ,  40.  ,   7.  ,  16.  ],
        [  0.  , 291.15,   0.  ,   0.  ,   1.  ,   2.  ,   2.  ],
        [  0.  , 289.64,   0.  ,   0.  ,   2.  ,   2.  ,   2.  ],
        [  0.  , 289.02,   0.  ,   0.  ,   1.  ,   5.  ,   4.  ],
        [  0.  , 288.78,   0.  ,   0.  ,   1.  ,   5.  ,   4.  ],
        [  0.  , 288.1 ,   0.  ,   0.  ,   1.  ,   2.  ,   2.  ],
        [  0.  , 288.15,   0.  ,   0.  ,   1.  ,   5.  ,   4.  ],
        [  0.  , 286.39,   0.  ,   0.  ,   1.  ,   5.  ,   4.  ],
        [  0.  , 286.35,   0.  ,   0.  ,   1.  ,   5.  ,   4.  ],
        [  0.  , 287.15,   0.  ,   0.  ,  90.  ,   5.  ,   4.  ],
        [  0.  , 289.34,   0.  ,   0.  ,   6.  ,   4.  ,   3.  ],
        [  0.  , 292.36,   0.  ,   0.  ,   6.  ,   4.  ,   3.  ],
        [ 

In [43]:

# Switch the model to evaluation mode
model.eval()

# Create empty lists to store inputs and predictions
all_inputs = []
all_predictions = []
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=256, shuffle=False, collate_fn=collate_fn, drop_last=False)
with torch.no_grad():  # Disable gradient calculation for inference
    for inputs, logits, lengths in test_dataloader:
        inputs = inputs.to('cuda')
        
        # Forward pass to get predictions
        outputs = model(inputs, lengths)
        
        # Store the predictions and inputs (moving to CPU if necessary)
        all_predictions.append(outputs.flatten().cpu().numpy())
        # all_inputs.append(inputs.cpu().numpy())

# Concatenate all predictions and inputs
# all_inputs = np.concatenate(all_inputs, axis=0)
all_predictions = np.concatenate(all_predictions, axis=0)

# Create a DataFrame to save results
# df_results = pd.DataFrame(all_inputs, columns=[f"feature_{i}" for i in range(all_inputs.shape[1])])
df_results = pd.DataFrame(all_predictions, columns=['Traffic Vol'])

# Save to CSV
df_results.to_csv("test_predictions.csv", index=False)


/tmp/ipykernel_3632/1669368327.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = self.y_seq[index]
